### Master of Applied Artificial Intelligence

**Course: TC5035 - Proyecto Integrador**

<img src="https://github.com/Medicenchapin/Proyecto-Integrador/blob/main/assets/logo.png?raw=1" alt="Image Alt Text" width="500"/>


**Baseline model**

Tutor: Dr. Horario Martinez Alfaro


Team members:
* Ignacio Jose Aguilar Garcia - A00819762
* Alejandro Calderon Aguilar - A01795353
* Ricardo Mar Cupido - A01795394

In [9]:
import sys
sys.path.append('../')

In [18]:
import importlib, scripts.helpers as hp
importlib.reload(hp)
from scripts.helpers import build_customer_prompt, build_global_context
import pandas as pd
import requests
import os


In [6]:
df = pd.read_parquet("../data/campaign_candidates_final.parquet")

In [7]:
df.head()

,state_name,previous_classification,previous_calls,client_age,network_age_years,banking,arpu_90_days,minutes_in,validity_average,average_performance,...,plan_postpaid,sn_banking,digital_index_mean,connected_days,charged_days,apps_days,music_gb,proba,sample_idx,drivers
0,GUATEMALA,NEW CLIENT,0,28.0,3.90,1,214.18,27.93,20.23,0.46,...,0.67,1.00,0.00,91,91,0,5.946,0.714486,0,"[{'feature': 'contacts', 'impact': -0.54814469..."
1,GUATEMALA,NEW CLIENT,0,40.0,0.20,1,135.39,116.22,1.26,0.68,...,0.23,0.46,0.00,67,62,2,0.000,0.620556,6,"[{'feature': 'network_age_years', 'impact': 0...."
2,GUATEMALA,NEW CLIENT,0,19.0,0.66,0,92.58,48.77,3.14,0.61,...,0.75,1.00,0.00,76,55,4,0.031,0.787552,10,"[{'feature': 'contacts', 'impact': -0.42579272..."
3,SAN MARCOS,NEW CLIENT,0,20.0,0.34,1,175.49,83.58,4.00,0.25,...,0.12,0.56,0.00,91,59,30,0.000,0.654747,12,"[{'feature': 'client_age', 'impact': 0.4658642..."
4,SAN MARCOS,NEW CLIENT,0,NaN,1.90,1,97.54,51.89,8.36,0.77,...,0.54,0.38,0.29,90,77,11,0.000,0.708033,26,"[{'feature': 'plan_postpaid', 'impact': 0.5166..."


## feature playbook 

In [20]:
# === 2) Mapeo opcional de features a “insights” de negocio (ajústalo a tus columnas) ===
FEATURE_PLAYBOOK = {
    "state_name": "Customer’s geographic region (e.g., GUATEMALA) or state. Use it to mention local availability or coverage.",
    "previous_classification": "Previous commercial label (e.g., NEW_CLIENT, NOT_INTERESTED, NOT EFFECTIVE). Adjust tone accordingly: onboarding, retention, or reactivation.",
    "previous_calls": "Number of previous customer calls or interactions. High value may indicate interest or friction; tailor the speech based on SHAP direction.",
    "client_age": "Customer’s age in years. Avoid demographic bias; use only to adapt communication tone if needed.",
    "network_age_years": "Years since the customer joined the network or service. Longer tenure suggests loyalty but possible reactivation opportunity.",
    "banking": "Binary flag indicating if the customer uses banking services. If yes, highlight convenience and trust; if no, keep it neutral.",
    "arpu_90_days": "Average revenue per user in the last 90 days. High ARPU means active/high-value user; low ARPU suggests opportunity to upsell or re-engage.",
    "minutes_in": "Incoming call minutes in the last period. Reflects engagement in voice usage.",
    "validity_average": "Average number of valid or active days in the plan. High validity shows consistency; low validity may indicate sporadic usage.",
    "average_performance": "Average service quality or network performance score. Low values may cause dissatisfaction; reinforce reliability or improvement initiatives.",
    "start_using_months": "Months since the customer started using the service. Long-time users can be re-engaged with loyalty or renewal offers.",
    "contacts": "Total previous contacts with the company. If high and SHAP is negative, reduce friction and simplify processes.",
    "high_frequency_contacts": "High-frequency contact flag. Frequent contacts can indicate interest or unresolved issues; address accordingly.",
    "plan_postpaid": "Indicates if the customer has a postpaid plan (1=yes, 0=no). Postpaid customers value premium experience, stability, and reliability.",
    "sn_banking": "Social or banking affinity indicator (internal feature). Use only if relevant; avoid assumptions if unclear.",
    "digital_index_mean": "Average digital engagement score. High score: emphasize digital channels, app self-service, and convenience.",
    "connected_days": "Number of days the customer was connected during the observed period. High value: active user; low: re-engagement opportunity.",
    "charged_days": "Number of days with active balance or successful billing. Shows payment activity or regular top-ups.",
    "apps_days": "Days with app usage activity. Low value suggests promoting app benefits or features.",
    "music_gb": "Amount of mobile data (in GB) used for music streaming. Zero usage = opportunity for 'music without data' offer.",
    "sale": "Target label (1=sale, 0=no sale). Do not include in prompts or explanations to avoid data leakage.",
}

## Build global prompt

In [40]:
global_prompt = build_global_context(df=df, feature_playbook=FEATURE_PLAYBOOK)
print(global_prompt)

You are helping generate sales guidance for a prepaid telecom campaign.

    We have a machine learning model that predicts the probability that a customer will accept an offer (sale = 1).
    The model was trained on historical customer behavior and engagement indicators. 
    Higher score means the customer is more likely to buy if contacted.

    The model relies on multiple behavioral and account features. Below are the most influential features overall (averaged across customers), and what they represent:

    - plan_postpaid: Indicates if the customer has a postpaid plan (1=yes, 0=no). Postpaid customers value premium experience, stability, and reliability.
- music_gb: Amount of mobile data (in GB) used for music streaming. Zero usage = opportunity for 'music without data' offer.
- network_age_years: Years since the customer joined the network or service. Longer tenure suggests loyalty but possible reactivation opportunity.
- contacts: Total previous contacts with the company. If

## Build prompt query by customer

In [41]:
idx = 26
row = df.loc[idx]

prompt_for_customer = build_customer_prompt(
    row=row,
    driver_list=row["drivers"],
    extra_context_cols=[
        "state_name",
        "previous_classification",
        "arpu_90_days",
        "network_age_years",
        "connected_days",
    ]
)

print(prompt_for_customer)

We are preparing a telemarketing/sales pitch for a prepaid mobile customer.

    Predicted probability of accepting the offer: 76.91%

    Relevant context for this customer:
    state_name = GUATEMALA
previous_classification = NEW CLIENT
arpu_90_days = 82.08
network_age_years = 8.870000000000001
connected_days = 90

    The following factors were most influential in predicting that this customer is likely to accept an offer:
    - plan_postpaid: value=1.6242012070997849, impact=+0.412
- client_age: value=-1.3159578561183756, impact=+0.217
- music_gb: value=0.5809999999999998, impact=+0.195
- previous_classification_NEW CLIENT: value=1.0, impact=+0.123
- previous_calls: value=-1.1469215649617823, impact=+0.115

    Task:
    1. Explain, in plain language, why this customer might respond positively.
    2. Suggest how an agent should position the offer (tone, focus, what to mention).
    3. Keep it short and actionable, as guidance for a call center agent.
    4. Do NOT mention 'model',

In [38]:

import requests
import os
import ollama

OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434/api/generate")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "mistral")

def call_llm(global_context: str, customer_prompt: str) -> str:
    full_prompt = (
        f"System:\n{global_context}\n\n"
        f"User:\n{customer_prompt}\n\n"
        "Assistant:"
    )
    
    full_prompt = [
        {'role': 'system', 'content': global_context},
        {'role': 'user', 'content': customer_prompt}
    ]



    # messages = [{'role': 'user', 'content': 'Tell me a story about a brave knight.'}]
    for chunk in ollama.chat('koesn/mistral-7b-instruct', messages=full_prompt, stream=True):
        print(chunk['message']['content'], end='', flush=True)
    # return out.strip()

In [42]:
call_llm(global_context=global_prompt, customer_prompt=prompt_for_customer)

This customer is likely to accept an offer due to their postpaid plan background and high data usage for music. Postpaid customers generally appreciate a premium experience, which our offer might deliver. Additionally, they consume quite a bit of mobile data for music. If the offer includes attractive deals or features related to music streaming, it could be particularly appealing to them.

In communicating with this customer, emphasize the benefits of our new offer and how it can enhance their mobile experience, especially when it comes to music enjoyment. Be sure to mention any exclusive deals or added features tailored to music enthusiasts. Maintain a friendly and helpful tone, making the conversation as engaging and personalized as possible.